In [21]:
import os
import requests
import random
import pickle
import networkx as nx
%matplotlib inline

KEY2 = '91mTqXFEeSaCIbjC0fuaB1RABk4HANqS4I91qRBN'
KEY = 'flLQZGa0kx8opciUA6M7R9iLPwfFPj5yFKKlCiVb'

MOST_RECENT_HR_BILL = 6536 #1490
MOST_RECENT_S_BILL = 3548  #606
CONGRESS = 114

In [14]:
# import re 
# from lxml import html
# # Create list of bills that passed at least one chamber of Congress

# temp = set()
# page = requests.get('https://www.congress.gov/search?q={%22type%22:%22bills%22,%22source%22:%22legislation%22,%22congress%22:%22114%22,%22bill-status%22:%22passed-one%22,%22chamber%22:%22Senate%22}&pageSize=250&page=1')
# tree = html.fromstring(page.content)
# bills = tree.xpath('//span[@class="result-heading"]//a/text()')
# temp |= set(bills)

# senate_passed_one_house = set()
# for bill in temp:
#     senate_passed_one_house.add(int(''.join(list(filter(str.isdigit, bill)))))

# temp = set()    
# for i in range(1, 10):
#     page = requests.get('https://www.congress.gov/search?pageSort=documentNumber:asc&q={%22congress%22:%22114%22,%22type%22:%22bills%22,%22bill-status%22:[%22passed-one%22],%22chamber%22:%22House%22}&pageSize=250&page=' + str(i))
#     tree = html.fromstring(page.content)
#     bills = tree.xpath('//span[@class="result-heading"]//a/text()')
#     temp |= set(bills)
    
# hr_passed_one_house = set()
# for bill in temp:
#     hr_passed_one_house.add(int(''.join(list(filter(str.isdigit, bill)))))


In [28]:
def getJSON(webpage):
    headers = {
        'X-API-Key': KEY
    }
    r = requests.get(webpage, headers=headers)
    return r.json()

def getSponsors(bill_id, congress, progress = 'introduced'):
    '''Returns (PrimarySponsor, [cosponsors]) for specified bill_id and congress strings. Returns () if faulty'''
    '''progress = 'introduced', 'became law'                                                                '''
    edge = getJSON('https://api.propublica.org/congress/v1/' + str(congress) + '/bills/' + bill_id + '/cosponsors.json')
    toReturn = False
    if edge['status'] == 'OK':
        if progress =='introduced':
            toReturn = True
        elif progress == 'became law':
            toReturn = edge['results'][0]['latest_major_action'][:17] == 'Became Public Law'           
                
    if toReturn:
        return (edge['results'][0]['sponsor_id'], [cosponsor['cosponsor_id'] for cosponsor in edge['results'][0]['cosponsors']])
    else:
        return ()
    
def getHouseMembers(congress):
    '''takes congress session and Returns dictioanry of congress member ids with value 0'''
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    houseList = []
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],0) for rep in house_members['results'][0]['members']]
    return dict(houseList)

def getSenateMembers(congress):
    '''takes congress session and Returns dictioanry of congress member ids with value 0'''
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    senateList = []
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'],0) for senator in senate_members['results'][0]['members']]
    return dict(senateList)


def getNameIDPairing(congress):
    senate_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/senate/members.json')
    house_members = getJSON('https://api.propublica.org/congress/v1/'+ str(congress) +'/house/members.json')
    senateList, houseList = [],[]
    if senate_members['status'] == 'OK':
        senateList = [(senator['id'], senator['first_name'] + ' ' + senator['last_name'] + '(' + senator['party'] + ')') for senator in senate_members['results'][0]['members']]
    if house_members['status'] == 'OK':
        houseList = [(rep['id'],rep['first_name'] + ' ' + rep['last_name'] + '(' + rep['party'] + ')') for rep in house_members['results'][0]['members']]
    return dict(senateList + houseList)

def buildHouseGraph(members, congress, progress='introduced'):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in range(1, MOST_RECENT_HR_BILL+1):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress, progress)
        if edge: # not (,)
            for u in edge[1]:
                graph[u].append(edge[0])
        
    return graph

def buildSenateGraph(members, congress, progress='introduced'):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in range(1, MOST_RECENT_S_BILL+1):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress, progress)
        if edge: # not (,)
            for u in edge[1]:
                graph[u].append(edge[0])
        
    return graph

def buildHouseGraph_PassedAtLeastOneChamber(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in sorted(list(hr_passed_one_house)):
        print("At HR{0}".format(i))
        edge = getSponsors("hr{0}".format(i), congress)
        if edge: # not (,)
            for u in edge[1]:
                graph[u].append(edge[0])
        
    return graph

def buildSenateGraph_PassedAtLeastOneChamber(members, congress):
    graph = {}
    for m in members:
        graph[m] = []
    
    for i in sorted(list(senate_passed_one_house)):
        print("At S{0}".format(i))
        edge = getSponsors("s{0}".format(i), congress, progress)
        if edge: # not (,)
            for u in edge[1]:
                graph[u].append(edge[0])
        
    return graph


def ranker(members, graph, p=.15, iterations=1000000):
    assert len(graph) == len(members)
    for m in members:
        members[m] = 0

    member_list = list(graph.keys())
    current = random.choice(member_list)
    members[current] += 1
    for i in range(iterations):
        if len(graph[current]) == 0 or random.random() < p:
            current = random.choice(member_list)
        else:
            neighbors = graph[current]
            current = neighbors[random.randint(0, len(neighbors)-1)]
        members[current] += 1
    return members


def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [16]:
house_members = getHouseMembers(CONGRESS)
senate_members = getSenateMembers(CONGRESS)
names = getNameIDPairing(CONGRESS)

save_obj(house_members, 'house' + str(CONGRESS) + '_members')
save_obj(senate_members, 'senate' + str(CONGRESS) + '_members')
save_obj(names, 'congress' + str(CONGRESS) + '_names')

In [23]:
# Use all bills that were introduced

# unfilteredHouseGraph = buildHouseGraph(house_members, CONGRESS)
unfilteredSenateGraph = buildSenateGraph(senate_members, CONGRESS)
save_obj(unfilteredHouseGraph, 'house' + str(CONGRESS) + '_unfilteredgraph')
save_obj(unfilteredSenateGraph, 'senate' + str(CONGRESS) + '_unfilteredgraph')

# unfilteredHouseGraph = load_obj('house' + str(CONGRESS) + '_unfilteredgraph')
# unfilteredSenateGraph = load_obj('senate' + str(CONGRESS) + '_unfilteredgraph')

At S1
At S2
At S3
At S4
At S5
At S6
At S7
At S8
At S9
At S10
At S11
At S12
At S13
At S14
At S15
At S16
At S17
At S18
At S19
At S20
At S21
At S22
At S23
At S24
At S25
At S26
At S27
At S28
At S29
At S30
At S31
At S32
At S33
At S34
At S35
At S36
At S37
At S38
At S39
At S40
At S41
At S42
At S43
At S44
At S45
At S46
At S47
At S48
At S49
At S50
At S51
At S52
At S53
At S54
At S55
At S56
At S57
At S58
At S59
At S60
At S61
At S62
At S63
At S64
At S65
At S66
At S67
At S68
At S69
At S70
At S71
At S72
At S73
At S74
At S75
At S76
At S77
At S78
At S79
At S80
At S81
At S82
At S83
At S84
At S85
At S86
At S87
At S88
At S89
At S90
At S91
At S92
At S93
At S94
At S95
At S96
At S97
At S98
At S99
At S100
At S101
At S102
At S103
At S104
At S105
At S106
At S107
At S108
At S109
At S110
At S111
At S112
At S113
At S114
At S115
At S116
At S117
At S118
At S119
At S120
At S121
At S122
At S123
At S124
At S125
At S126
At S127
At S128
At S129
At S130
At S131
At S132
At S133
At S134
At S135
At S136
At S137
At S138
At S

In [29]:
# Ranking of Representatives based on cosponsorships of all bills introduced. 

ranking = ranker(house_members, unfilteredHouseGraph)

#Code below copied from trailer
member_ranks = [pair[0] for pair in sorted(house_members.items(), key = lambda item: item[1], reverse=True)]
top_score = house_members[member_ranks[0]]
for i in range(len(member_ranks)):
    print("%d %f: %s" % (i+1,house_members[member_ranks[i]]/top_score, names[member_ranks[i]]))

1 1.000000: Diane Black(R)
2 0.806247: Erik Paulsen(R)
3 0.758251: Sam Johnson(R)
4 0.740013: Brett Guthrie(R)
5 0.727239: Charles Boustany Jr.(R)
6 0.665067: Christopher Smith(R)
7 0.653769: Paul Gosar(R)
8 0.652293: Tom Price(R)
9 0.634941: Kevin Brady(R)
10 0.626228: Peter Roskam(R)
11 0.602525: Tom Reed(R)
12 0.587314: Lynn Jenkins(R)
13 0.573876: Todd Young(R)
14 0.556598: Carolyn Maloney(D)
15 0.546039: Matt Cartwright(D)
16 0.539688: Robert Goodlatte(R)
17 0.523518: Michael McCaul(R)
18 0.511187: Rosa DeLauro(D)
19 0.489995: Dave Reichert(R)
20 0.484605: Trent Franks(R)
21 0.483792: F. Sensenbrenner(R)
22 0.466219: Ted Poe(R)
23 0.458392: Pat Tiberi(R)
24 0.432770: Phil Roe(R)
25 0.431367: Edward Royce(R)
26 0.428635: Tim Walberg(R)
27 0.426567: Blaine Luetkemeyer(R)
28 0.417411: Pete Olson(R)
29 0.414310: David Cicilline(D)
30 0.413498: Mike Kelly(R)
31 0.412686: Bill Posey(R)
32 0.411283: Matt Salmon(R)
33 0.404932: Chris Gibson(R)
34 0.397549: Martha McSally(R)
35 0.394817: T

In [30]:
# Ranking of Senators based on cosponsorships of all bills introduced. 

ranking = ranker(senate_members, unfilteredSenateGraph)
member_ranks = [pair[0] for pair in sorted(senate_members.items(), key = lambda item: item[1], reverse=True)]
top_score = senate_members[member_ranks[0]]
for i in range(len(member_ranks)):
    print("%d %f: %s" % (i+1,senate_members[member_ranks[i]]/top_score, names[member_ranks[i]]))

1 1.000000: Orrin Hatch(R)
2 0.980173: Charles Grassley(R)
3 0.817104: John Thune(R)
4 0.694946: Mark Kirk(R)
5 0.690313: Benjamin Cardin(D)
6 0.661686: John Cornyn(R)
7 0.634209: Marco Rubio(R)
8 0.622751: Jerry Moran(R)
9 0.616573: Mike Lee(R)
10 0.609461: Rob Portman(R)
11 0.596423: Lamar Alexander(R)
12 0.594663: Roy Blunt(R)
13 0.564707: Sherrod Brown(D)
14 0.555332: Patty Murray(D)
15 0.554793: Kelly Ayotte(R)
16 0.553464: Dianne Feinstein(D)
17 0.546209: Ron Wyden(D)
18 0.541360: Susan Collins(R)
19 0.532129: Jeanne Shaheen(D)
20 0.512158: John Barrasso(R)
21 0.511584: Ted Cruz(R)
22 0.504508: Jeff Flake(R)
23 0.500485: Bob Casey(D)
24 0.473438: Bill Nelson(D)
25 0.473079: Michael Crapo(R)
26 0.463202: Johnny Isakson(R)
27 0.461837: Patrick Toomey(R)
28 0.458927: Richard Blumenthal(D)
29 0.453109: James Inhofe(R)
30 0.442369: Kirsten Gillibrand(D)
31 0.435401: Dean Heller(R)
32 0.434970: David Vitter(R)
33 0.429654: Rand Paul(R)
34 0.414568: Robert Menendez(D)
35 0.407528: Richa

In [26]:
# Use only bills that were passed into law

filteredByPassedLawHouse = buildHouseGraph(house_members, CONGRESS, progress='became law')
save_obj(filteredByPassedLawHouse, 'house' + str(CONGRESS) + '_filteredPassedLawHouse')
filteredByPassedLawSenate = buildSenateGraph(senate_members, CONGRESS, progress='became law')
save_obj(filteredByPassedLawSenate, 'senate' + str(CONGRESS) + '_filteredPassedLawSenate')

# filteredByPassedLawHouse = load_obj('house' + str(CONGRESS) + '_filteredPassedLawHouse')
# filteredByPassedLawSenate = load_obj(filteredByPassedLawSenate, 'senate' + str(CONGRESS) + '_filteredPassedLawSenate')

At HR1
At HR2
At HR3
At HR4
At HR5
At HR6
At HR7
At HR8
At HR9
At HR10
At HR11
At HR12
At HR13
At HR14
At HR15
At HR16
At HR17
At HR18
At HR19
At HR20
At HR21
At HR22
At HR23
At HR24
At HR25
At HR26
At HR27
At HR28
At HR29
At HR30
At HR31
At HR32
At HR33
At HR34
At HR35
At HR36
At HR37
At HR38
At HR39
At HR40
At HR41
At HR42
At HR43
At HR44
At HR45
At HR46
At HR47
At HR48
At HR49
At HR50
At HR51
At HR52
At HR53
At HR54
At HR55
At HR56
At HR57
At HR58
At HR59
At HR60
At HR61
At HR62
At HR63
At HR64
At HR65
At HR66
At HR67
At HR68
At HR69
At HR70
At HR71
At HR72
At HR73
At HR74
At HR75
At HR76
At HR77
At HR78
At HR79
At HR80
At HR81
At HR82
At HR83
At HR84
At HR85
At HR86
At HR87
At HR88
At HR89
At HR90
At HR91
At HR92
At HR93
At HR94
At HR95
At HR96
At HR97
At HR98
At HR99
At HR100
At HR101
At HR102
At HR103
At HR104
At HR105
At HR106
At HR107
At HR108
At HR109
At HR110
At HR111
At HR112
At HR113
At HR114
At HR115
At HR116
At HR117
At HR118
At HR119
At HR120
At HR121
At HR122
At HR123
A

In [31]:
# Ranking of Representatives based only on cosponsorships of bills that were actually signed into law. 

ranking = ranker(house_members, filteredByPassedLawHouse)
member_ranks = [pair[0] for pair in sorted(house_members.items(), key = lambda item: item[1], reverse=True)]
top_score = house_members[member_ranks[0]]
for i in range(len(member_ranks)):
    print("%d %f: %s" % (i+1,house_members[member_ranks[i]]/top_score, names[member_ranks[i]]))

1 1.000000: Sam Johnson(R)
2 0.801382: Bill Posey(R)
3 0.741576: Carolyn Maloney(D)
4 0.715829: Terri Sewell(D)
5 0.706899: Jeff Fortenberry(R)
6 0.590983: Brett Guthrie(R)
7 0.567227: Martha McSally(R)
8 0.548026: Seth Moulton(D)
9 0.507255: Edward Royce(R)
10 0.337657: Christopher Smith(R)
11 0.323222: Stacey Plaskett(D)
12 0.285604: Henry Cuellar(D)
13 0.280959: Kevin Brady(R)
14 0.263794: Lamar Smith(R)
15 0.195285: Stephen Lynch(D)
16 0.179583: Joseph Kennedy III(D)
17 0.175601: Rodney Davis(R)
18 0.163399: Pete Olson(R)
19 0.161423: Grace Meng(D)
20 0.154092: Vern Buchanan(R)
21 0.152478: José Serrano(D)
22 0.151679: Cynthia Lummis(R)
23 0.148104: Erik Paulsen(R)
24 0.132659: Madeleine Bordallo(D)
25 0.130411: Brad Ashford(D)
26 0.108088: David Rouzer(R)
27 0.106579: Mimi Walters(R)
28 0.098223: Jackie Walorski(R)
29 0.096368: Frank Pallone(D)
30 0.091059: Judy Chu(D)
31 0.090878: David Brat(R)
32 0.090742: Julia Brownley(D)
33 0.089264: Gus Bilirakis(R)
34 0.088449: Susan Davis(

In [33]:
# Ranking of Senators based only on cosponsorships of bills that were actually signed into law. 

ranking = ranker(senate_members, filteredByPassedLawSenate)
member_ranks = [pair[0] for pair in sorted(senate_members.items(), key = lambda item: item[1], reverse=True)]
top_score = senate_members[member_ranks[0]]
for i in range(len(member_ranks)):
    print("%d %f: %s" % (i+1,senate_members[member_ranks[i]]/top_score, names[member_ranks[i]]))

1 1.000000: John Cornyn(R)
2 0.713656: Roy Blunt(R)
3 0.683725: Orrin Hatch(R)
4 0.627329: Dianne Feinstein(D)
5 0.525480: Mazie Hirono(D)
6 0.492836: Benjamin Cardin(D)
7 0.421928: Sheldon Whitehouse(D)
8 0.320936: Patrick Leahy(D)
9 0.280296: Heidi Heitkamp(D)
10 0.278308: Rob Portman(R)
11 0.272240: Cory Gardner(R)
12 0.260410: Richard Burr(R)
13 0.246673: Lamar Alexander(R)
14 0.229938: Bob Corker(R)
15 0.222840: Mitch McConnell(R)
16 0.188237: Bob Casey(D)
17 0.175561: Kelly Ayotte(R)
18 0.174613: Roger Wicker(R)
19 0.155155: Ron Johnson(R)
20 0.151646: Patrick Toomey(R)
21 0.151014: Barbara Boxer(D)
22 0.150535: Ron Wyden(D)
23 0.144691: Susan Collins(R)
24 0.140102: Bill Nelson(D)
25 0.127548: Thomas Carper(D)
26 0.109742: Jon Tester(D)
27 0.103980: Al Franken(D)
28 0.088347: Thom Tillis(R)
29 0.087337: Mark Kirk(R)
30 0.086113: Amy Klobuchar(D)
31 0.085654: Deb Fischer(R)
32 0.073886: John Thune(R)
33 0.071550: Marco Rubio(R)
34 0.067675: Charles Grassley(R)
35 0.067502: Brian 

In [ ]:
# filteredPassedAtLeastOneChamberHouse = buildHouseGraph_PassedAtLeastOneChamber(house_members, CONGRESS)
# filteredPassedAtLeastOneChamberSenate = buildSenateGraph_PassedAtLeastOneChamber(senate_members, CONGRESS)
# save_obj(filteredPassedAtLeastOneChamberHouse, 'house' + str(CONGRESS) + '_filteredPassedAtLeastOneChamber_House')
# save_obj(filteredPassedAtLeastOneChamberSenate, 'senate' + str(CONGRESS) + '_filteredPassedAtLeastOneChamber_Senate')

# filteredPassedAtLeastOneChamberHouse = save_obj('house' + str(CONGRESS) + '_filteredPassedAtLeastOneChamber_House')
# filteredPassedAtLeastOneChamberSenate = save_obj('senate' + str(CONGRESS) + '_filteredPassedAtLeastOneChamber_Senate')

In [ ]:
# ranking = ranker(house_members, filteredPassedAtLeastOneChamberHouse)
# member_ranks = [pair[0] for pair in sorted(house_members.items(), key = lambda item: item[1], reverse=True)]
# top_score = house_members[member_ranks[0]]
# for i in range(len(member_ranks)):
#     print("%d %f: %s" % (i+1,house_members[member_ranks[i]]/top_score, names[member_ranks[i]]))

In [ ]:
# ranking = ranker(senate_members, filteredPassedAtLeastOneChamberSenate)
# member_ranks = [pair[0] for pair in sorted(senate_members.items(), key = lambda item: item[1], reverse=True)]
# top_score = senate_members[member_ranks[0]]
# for i in range(len(member_ranks)):
#     print("%d %f: %s" % (i+1,senate_members[member_ranks[i]]/top_score, names[member_ranks[i]]))